# SQL DB

Practice dataset csv file transformed into a sql database

In [1]:
#imports
import sqlite3
import numpy as np
import pandas as pd 
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
#Create a database and connect with sqlite3

DBPATH = 'results.db'
conn = sqlite3.connect(DBPATH)
cur = conn.cursor()

In [3]:
#Read csv file into a DataFrame
csvfile ='results.csv'
resdf = pd.read_csv(csvfile)

#reviewing dataframe and loaded data
resdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Columns: 129 entries, Unnamed: 0 to Postn Lang 1 Rd
dtypes: float64(4), object(125)
memory usage: 751.9+ KB


In [9]:
#Display the dataframe to view the output
resdf=resdf.drop(['Unnamed: 0'], axis=1)
resdf.head(3)


,ID-ORDER,datetime,Q1a_lang1,Q2-1_Meetings_Freq,Q2-1_Meetings_Import,Q2-1_Meetings_Eng,Q2-1_Meetings_LES,Q2-2_PreparedStats_Freq,Q2-2_PreparedStats_Import,Q2-2_PreparedStats_Eng,...,Emp Grade,EmpGradeRank,Lang Cd,Emp Speak Proficiency 1,Speak Proficiency Plus,Emp Read Proficiency 1,Read Proficiency Plus,Postn Lang 1 Sp/Rd,Postn Lang 1 Sp,Postn Lang 1 Rd
0,NA00011,2017-09-05 21:53:00,Chinese (Mandarin),Weekly,Important,yes,no,Never - not applicable,Not important,yes,...,01,SENIOR,CM,3.0,NaN,3.0,NaN,CM,3,3
1,NA00012,2017-09-05 15:29:00,Romanian,Weekly,Important,yes,yes,Every few months,Important,yes,...,04,ENTRY,RQ,3.0,NaN,3.0,NaN,RQ,3,3
2,NA00013,2017-09-05 10:01:00,Spanish,Monthly,Very important,NaN,yes,Every few months,Slightly important,no,...,03,MID,QB,2.0,NaN,2.0,NaN,QB,2,2


In [ ]:
#Converts df slice to demographics sql table
resdf.iloc[:,114:129].to_sql('demotabl',conn)

In [22]:
#Using cursor to connect and query the newly created table
cur.execute('SELECT EmpGradeRank, count(EmpGradeRank) FROM demotabl WHERE EmpGradeRank="SENIOR";')
cur.fetchone()

('SENIOR', 92)

In [ ]:
#Creating slice from the df for the second table
survey = resdf.iloc[:,:113]
survey.insert(2,'LangCd',resdf.iloc[:,120])
survey.to_sql('surveytabl',conn)

In [23]:
#Select lang field and counting returned values from the table
cur.execute('SELECT LangCd ,count(*) FROM surveytabl WHERE LangCd="QB";')
cur.fetchall()

[('QB', 225)]

In [24]:
#Select all the non-null comment fields from the table
cur.execute('SELECT count(Comments_Ratings) FROM surveytabl WHERE Comments_Ratings is not null;')
cur.fetchall()

[(327,)]

In [25]:
#Check the number of tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('demotable',), ('demotabl',), ('survey',), ('surveytabl',)]


In [26]:
#Alternate method for reading sql with pandas
engine = create_engine('sqlite:///results.db')
pd.read_sql_query('SELECT * FROM demotabl LIMIT 5;',engine)

,index,PostnRelabel,Bureau,Org Name,Country,Emp Grade,EmpGradeRank,Lang Cd,Emp Speak Proficiency 1,Speak Proficiency Plus,Emp Read Proficiency 1,Read Proficiency Plus,Postn Lang 1 Sp/Rd,Postn Lang 1 Sp,Postn Lang 1 Rd
0,0,Operations,EAP,BEIJING,PEOPLES REP OF CHINA,01,SENIOR,CM,3.0,None,3.0,None,CM,3,3
1,1,Customer Service,EUR,BUCHAREST,ROMANIA,04,ENTRY,RQ,3.0,None,3.0,None,RQ,3,3
2,2,Security,WHA,PANAMA,PANAMA,03,MID,QB,2.0,None,2.0,None,QB,2,2
3,3,Manager,EUR,PARIS,FRANCE,05,ENTRY,FR,2.0,None,2.0,None,FR,2,2
4,4,Customer Service,EUR,ISTANBUL,TURKEY,03,MID,TU,3.0,None,3.0,None,TU,3,3


In [21]:
#Return to excel; you can save the newly created tables in a new excel file in separate tabs 
writer= pd.ExcelWriter('results.xlsx')
resdf.iloc[:,114:129].to_excel(writer,'Demo')
survey.to_excel(writer,'Survey')
writer.save()